In [4]:
#import pandas then load file DONE
#describe and columns to understand the data DONE
#check columns with 1 unique value to remove DONE
#create sales column (sold x price) DONE
#total sales DONE
#create share DONE
#top 10 items DONE
#categories shares DONE
#remove unnecessary columns in categories df DONE
#plot some charts for category DONE
#count blanks across all columns DONE
#replace blanks with 0 or NA DONE
#include "blank" brands data after pivoting DONE
#Learn groupby DONE
#learn concat DONE
#top 5 brands for each category DONE
#calculate share of segment DONE
#adjust time column

In [1]:
import pandas as pd

original = pd.read_excel(r"C:\Users\Ahmed\Desktop\Job Hunt\TMO\MIE-sample-data ORIGINAL.xlsx", index_col=0)

original.describe()

original.columns

original.platform.unique()
original.category.unique()
original.country.unique()
original.shop_id.unique()
original.shop_name.unique()

original.drop(columns=['country', 'platform'], axis=1, inplace=True)

original['sales'] = original['sold'] * original['price']

original.columns

original['found_brand'].fillna('Other Brand', inplace=True)

missing_zero_values_table(original)

original.sales.sum()

original['share'] = (original['sales'] / original.sales.sum()) * 100

original.share.sum()

original.sort_values('share', ascending=False).head(10)

category = pd.pivot_table(original, index='category', aggfunc=['sum', 'mean'])

category.index

category.columns

category_clean = category.drop(columns=[('sum', 'price'), ('sum', 'rating'), ('sum', 'itemId'), ('sum', 'shop_id'), ('sum', 'sold'), ('mean', 'itemId'), ('mean', 'share'), ('mean', 'shop_id'), ('mean', 'sold')], axis=1)

category_clean.sort_values(('sum', 'share'), ascending=False, inplace=True)

original.sales.plot()

category_clean[('sum', 'share')].plot()

category_clean.columns = ['_'.join(col) for col in category_clean.columns]

category_clean.sort_values('sum_share', ascending=False, inplace=True)

category_clean.rename({'sum_share': 'share', 'sum_sales': 'sales'}, axis=1, inplace=True)

category_clean.share.plot(kind='bar')

brand_category = pd.pivot_table(original, index=('category', 'found_brand'), aggfunc=['sum', 'mean'])

brand_category.columns = ['_'.join(col) for col in brand_category.columns]

brand_category.drop(columns=['sum_itemId', 'sum_price', 'sum_rating', 'sum_shop_id', 'mean_itemId', 'mean_share', 'mean_shop_id'], axis=1, inplace=True)

brand_category.sort_values('sum_share', ascending=False)

brand_category.sort_values(['category','sum_share'], ascending=[True,False])

brand_category.loc['Beauty', :].sum_sales.sum()

top_brands = (brand_category
     .groupby(level=0, as_index=False)
     .apply(lambda x: x.head(5) if len(x) >= 5 else x.head(0))
     .reset_index(level=0, drop=True)
)

In [13]:
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

missing_zero_values_table(original)

Your selected dataframe has 15 columns and 8970 Rows.
There are 1 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
found_brand,0,5263,58.7,5263,58.7,object
